In [48]:
import glob
import pandas as pd
import re

In [13]:
import os
import json

config_file = os.path.join(os.path.abspath(''), os.pardir, 'scripts', 'config.json')
with open(config_file) as json_data:
    config = json.load(json_data)

fmriprep_dir = config['fmriprep_dir']

In [50]:
os.listdir(fmriprep_dir)
glob_str = os.path.join(fmriprep_dir, 'sub-*', 'func', '*_confounds.tsv')
counfound_files = glob.glob(glob_str)

to_check = []

for confound_file in counfound_files:
    df = pd.read_csv(confound_file, delimiter='\t')
    fd = df['FramewiseDisplacement']
    
    if (any(fd>2)):
        m = re.search('(sub-\d+_task-MGT_run-\d+)', confound_file)
        if m:
            sub_run = m.group(1)
        else:
            raise Exception('sub_run not found in ' + confound_file)
        to_check.append(sub_run)
        
print(to_check)

['sub-013_task-MGT_run-04', 'sub-016_task-MGT_run-01', 'sub-016_task-MGT_run-03', 'sub-016_task-MGT_run-04', 'sub-018_task-MGT_run-01', 'sub-018_task-MGT_run-02', 'sub-018_task-MGT_run-03', 'sub-018_task-MGT_run-04', 'sub-022_task-MGT_run-02', 'sub-026_task-MGT_run-01', 'sub-026_task-MGT_run-04', 'sub-030_task-MGT_run-01', 'sub-030_task-MGT_run-02', 'sub-030_task-MGT_run-03', 'sub-030_task-MGT_run-04', 'sub-036_task-MGT_run-02', 'sub-037_task-MGT_run-03', 'sub-068_task-MGT_run-02', 'sub-088_task-MGT_run-02', 'sub-088_task-MGT_run-03', 'sub-088_task-MGT_run-04', 'sub-089_task-MGT_run-03', 'sub-093_task-MGT_run-04', 'sub-100_task-MGT_run-02', 'sub-100_task-MGT_run-04', 'sub-106_task-MGT_run-03', 'sub-106_task-MGT_run-04', 'sub-110_task-MGT_run-01', 'sub-110_task-MGT_run-03', 'sub-110_task-MGT_run-04', 'sub-116_task-MGT_run-01', 'sub-116_task-MGT_run-02', 'sub-116_task-MGT_run-03', 'sub-116_task-MGT_run-04', 'sub-120_task-MGT_run-02']
